In [2]:
# usando autoreload no notebook
%load_ext autoreload
%autoreload 2

In [3]:
# Importando módulos

from hastads_function import *
from common_module_functions import *
from utils import *

## Utilizando o ataque de módulo comum

### Primeiro caso: p e q pequenos

In [4]:
# Criando dicionário para juntar os 2 arquivos de texto do caso Common Modulus Attack
email_common_modulus = {
    'email1.eml': {},
    'email2.eml': {}
}

for key in email_common_modulus:
    email_common_modulus[key] = ler_arquivo_eml_e_converter_para_lista(key)

In [5]:
# Exemplo de uso

from common_module_functions import PAndQFoundException

p_e_q_encontrados = False

n = email_common_modulus.get('email1.eml').get('N')
e1 = email_common_modulus.get('email1.eml').get('e')
e2 = email_common_modulus.get('email2.eml').get('e')
corpo_c1 = email_common_modulus.get('email1.eml').get('corpo')
corpo_c2 = email_common_modulus.get('email2.eml').get('corpo')

corpo_descriptografado = []
assunto_descriptografado = []

# Realiza o ataque no corpo dos emails
try:
    for c1, c2 in zip(corpo_c1, corpo_c2):
        caractere = attack(c1, c2, e1, e2, n)
        caractere = int_to_str(caractere)
        corpo_descriptografado.append(caractere)
except PAndQFoundException as err:
    print(f"Os fatores primos de N são p={err.p} e q={err.q}")
    p_e_q_encontrados = True
    p=err.p
    q=err.q

# Realiza o ataque no assunto dos emails
assunto_c1 = email_common_modulus.get('email1.eml').get('assunto')
assunto_c2 = email_common_modulus.get('email2.eml').get('assunto')

if not p_e_q_encontrados:
    try:
        for c1, c2 in zip(assunto_c1, assunto_c2):
            caractere = attack(c1, c2, e1, e2, n)
            caractere = int_to_str(caractere)
            assunto_descriptografado.append(caractere)

    except PAndQFoundException as err:
            print(f"Os fatores primos de N são p={err.p} e q={err.q}")
            p_e_q_encontrados = True
            p=err.p
            q=err.q
        

Não foi possível encontrar o inverso modular de 114 modulo 779, logo, eles possuem um fator em comum.
Os fatores primos de N são p=19 e q=41


In [6]:
# Imprime a mensagem descriptografada
if p_e_q_encontrados:
    corpo_descriptografado = []
    assunto_descriptografado = []

    for c1 in corpo_c1:
        m = decrypt_pq(c1, p, q, e1)
        m = chr(m)
        corpo_descriptografado.append(m)

    for c1 in assunto_c1:
        m = decrypt_pq(c1, p, q, e1)
        m = chr(m)
        assunto_descriptografado.append(m)

print("Email Descriptografado!")
print("Assunto: ", end='')
for i in assunto_descriptografado:
    print(i, end='')
print('\nCorpo: ', end='')
for i in corpo_descriptografado:
    print(i, end='')

Email Descriptografado!
Assunto: duvido vc descobrir
Corpo: algebra é muito legal

### Segundo caso: p e q grandes:

In [17]:
# Criando dicionário para juntar os 2 arquivos de texto do caso Common Modulus Attack
email_common_modulus = {
    'common_module1.eml': {},
    'common_module2.eml': {}
}

for key in email_common_modulus:
    email_common_modulus[key] = ler_arquivo_eml_e_converter_para_lista(key, False)

In [25]:
# Exemplo de uso

from common_module_functions import PAndQFoundException

p_e_q_encontrados = False

n = email_common_modulus.get('common_module1.eml').get('N')
e1 = email_common_modulus.get('common_module1.eml').get('e')
e2 = email_common_modulus.get('common_module2.eml').get('e')
corpo_c1 = email_common_modulus.get('common_module1.eml').get('corpo')
corpo_c2 = email_common_modulus.get('common_module2.eml').get('corpo')

corpo_descriptografado = []

# Realiza o ataque no corpo dos emails
try:
    for c1, c2 in zip(corpo_c1, corpo_c2):
        caractere = attack(c1, c2, e1, e2, n)
        caractere = chr(caractere)
        corpo_descriptografado.append(caractere)
except PAndQFoundException as err:
    print(f"Os fatores primos de N são p={err.p} e q={err.q}")
    p_e_q_encontrados = True
    p=err.p
    q=err.q
    
# Imprime a mensagem descriptografada
if not p_e_q_encontrados:
    print("Email Descriptografado!")
    print("Corpo: ", end='')
    for i in corpo_descriptografado:
        print(i, end='')
else:
    corpo_descriptografado = []

    for c1 in corpo_c1:
        m = decrypt_pq(c1, p, q, e1)
        m = chr(m)
        corpo_descriptografado.append(m)

    print("Email Descriptografado!")
    print("Corpo: ", end='')
    for i in corpo_descriptografado:
        print(i, end='')

Email Descriptografado!
Corpo: duvido você descobrir

## Utilizando o ataque de Hastad's

### Primeiro caso: um par de N's não coprimos

In [7]:
# Criando dicionário para juntar os 3 arquivos de texto do caso Hastads Broadcast Attack
email_hastads = {
    'email_pessoa1.eml': [],
    'email_pessoa2.eml': [],
    'email_pessoa3.eml': []
}

for key in email_hastads:
    email_hastads[key] = ler_arquivo_eml_e_converter_para_lista(key)

In [8]:
# Descripitografando a mensagem dos emails

n1 = email_hastads.get('email_pessoa1.eml').get('N')
n2 = email_hastads.get('email_pessoa2.eml').get('N')
n3 = email_hastads.get('email_pessoa3.eml').get('N')

corpo_c1 = email_hastads.get('email_pessoa1.eml').get('corpo')
corpo_c2 = email_hastads.get('email_pessoa2.eml').get('corpo')
corpo_c3 = email_hastads.get('email_pessoa3.eml').get('corpo')

corpo_descriptografado = []
assunto_descriptografado = []
p_e_q_encontrados = False
modulus = [n1, n2, n3]

for c1, c2, c3 in zip(corpo_c1, corpo_c2, corpo_c3):
    ciphertexts = [c1, c2, c3]
    try: 
        caractere = hastads_broadcast_attack(ciphertexts, modulus)
        caractere = chr(caractere)
        corpo_descriptografado.append(caractere)
    except PAndQFoundException as err:
        print(f"Os fatores primos de N são p:{err.p} e q:{err.q}")
        p_e_q_encontrados = True
        p = err.p
        q = err.q
        break

if not p_e_q_encontrados:
    for c1, c2, c3 in zip(assunto_c1, assunto_c2, assunto_c3):
        ciphertexts = [c1, c2, c3]
        try: 
            caractere = hastads_broadcast_attack(ciphertexts, modulus)
            caractere = chr(caractere)
            assunto_descriptografado.append(caractere)
        except PAndQFoundException as err:
            print(f"Os fatores primos de N são p:{err.p} e q:{err.q}")
            p_e_q_encontrados = True
            p = err.p
            q = err.q
            break


Os fatores primos de N são p:29 e q:251


In [9]:
e = 3

if p_e_q_encontrados:
    corpo_descriptografado = []
    assunto_descriptografado = []

    # descobrindo qual dos n's é o produto de p e q para saber qual corpo e assunto descriptografar
    if p*q == n1:
        corpo_c = corpo_c1
        assunto_c = email_hastads.get('email_pessoa1.eml').get('assunto')
    elif p*q == n2:
        corpo_c = corpo_c2
        assunto_c = email_hastads.get('email_pessoa2.eml').get('assunto')
    elif p*q == n3:
        corpo_c = corpo_c3
        assunto_c = email_hastads.get('email_pessoa3.eml').get('assunto')

    for c in corpo_c:
        m = decrypt_pq(c, p, q, e)
        m = chr(m)
        corpo_descriptografado.append(m)

    for c in assunto_c:
        m = decrypt_pq(c, p, q, e)
        m = chr(m)
        assunto_descriptografado.append(m)

print("Email Descriptografado!")
print("Assunto: ", end='')
for i in assunto_descriptografado:
    print(i, end='')
print('\nCorpo: ', end='')
for i in corpo_descriptografado:
    print(i, end='')

    

Email Descriptografado!
Assunto: agora esse é impossível
Corpo: pois não é que vc descobriu mesmo

### Segundo caso: N's coprimos

In [27]:
# Criando dicionário para juntar os 3 arquivos de texto do caso Hastads Broadcast Attack
email_hastads = {
    'hastad_pessoa1.eml': [],
    'hastad_pessoa2.eml': [],
    'hastad_pessoa3.eml': []
}

for key in email_hastads:
    email_hastads[key] = ler_arquivo_eml_e_converter_para_lista(key, False)

In [29]:
# Descripitografando a mensagem dos emails

n1 = email_hastads.get('hastad_pessoa1.eml').get('N')
n2 = email_hastads.get('hastad_pessoa2.eml').get('N')
n3 = email_hastads.get('hastad_pessoa3.eml').get('N')

corpo_c1 = email_hastads.get('hastad_pessoa1.eml').get('corpo')
corpo_c2 = email_hastads.get('hastad_pessoa2.eml').get('corpo')
corpo_c3 = email_hastads.get('hastad_pessoa3.eml').get('corpo')

corpo_descriptografado = []
p_e_q_encontrados = False
modulus = [n1, n2, n3]

for c1, c2, c3 in zip(corpo_c1, corpo_c2, corpo_c3):
    ciphertexts = [c1, c2, c3]
    try: 
        caractere = hastads_broadcast_attack(ciphertexts, modulus)
        caractere = chr(caractere)
        corpo_descriptografado.append(caractere)
    except PAndQFoundException as err:
        print(f"Os fatores primos de N são p:{err.p} e q:{err.q}")
        p_e_q_encontrados = True
        p = err.p
        q = err.q
        break

In [31]:
if not p_e_q_encontrados:
    print("Email Descriptografado!")
    print("Corpo: ", end='')
    for i in corpo_descriptografado:
        print(i, end='')

Email Descriptografado!
Corpo: agora esse é impossível